In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import io
import string
import re
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
os.chdir("./drive/My Drive/CK Bigdata/pet-translate-master/dataset/")

In [ ]:
en_filename = "train.en.txt"
vi_filename = "train.vi.txt"

raw_en_lines = open(en_filename, encoding='utf-8').read().strip().split("\n")
raw_vi_lines = open(vi_filename, encoding='utf-8').read().strip().split("\n")

In [ ]:
exclude = list(string.punctuation) + list(string.digits)

def preprocess(sentence):
    sent = sentence.lower()
    sent = sent.strip()
    sent = re.sub("'", " ", sent)
    sent = re.sub("\s+", " ", sent)
    sent = ''.join([char for char in sent if char not in exclude])
    sent = "<start> " + sent + " <end>"
    return sent

In [ ]:
en_lines = []
vi_lines = []
min_len, max_len = 10, 14

for eline, vline in zip(raw_en_lines, raw_vi_lines):
    eline = preprocess(eline)
    vline = preprocess(vline)
    if min_len < len(eline.split(" ")) < max_len and min_len < len(vline.split(" ")) < max_len:
        en_lines.append(eline)
        vi_lines.append(vline)
        
print(len(en_lines))

6016


In [ ]:
class Language():
    def __init__(self, lines):
        self.lines = lines
        self.word2id = {}
        self.id2word = {}
        self.vocab = set()
        self.max_len = 0
        self.min_len = 0
        self.vocab_size = 0
        self.init_language_params()

    def init_language_params(self):
        for line in self.lines:
            self.vocab.update(line.split(" "))
        self.word2id['<pad>'] = 0
        for id, word in enumerate(self.vocab):
            self.word2id[word] = id + 1
        for word, id in self.word2id.items():
            self.id2word[id] = word
        self.max_len = max([len(line.split(" ")) for line in self.lines])
        self.min_len = min([len(line.split(" ")) for line in self.lines])
        self.vocab_size = len(self.vocab) + 1
            
    def sentence_to_vector(self, sent):
        return np.array([self.word2id[word] for word in sent.split(" ")])
            
    def vector_to_sentence(self, vector):
        return " ".join([self.id2word[id] for id in vector])

In [ ]:
inp_lang = Language(en_lines)
tar_lang = Language(vi_lines)
print(inp_lang.max_len, inp_lang.min_len)
print(tar_lang.max_len, tar_lang.min_len)

inp_vector = [inp_lang.sentence_to_vector(line) for line in inp_lang.lines]
tar_vector = [tar_lang.sentence_to_vector(line) for line in tar_lang.lines]

inp_tensor = tf.keras.preprocessing.sequence.pad_sequences(inp_vector, inp_lang.max_len, padding='post')
tar_tensor = tf.keras.preprocessing.sequence.pad_sequences(tar_vector, tar_lang.max_len, padding='post')
print(inp_tensor.shape, tar_tensor.shape)

13 11
13 11
(6016, 13) (6016, 13)


In [ ]:

x_train, x_val, y_train, y_val = train_test_split(inp_tensor, tar_tensor, test_size=0.3)

BATCH_SIZE = 32
BUFFER_SIZE = x_train.shape[0]
N_BATCH = BUFFER_SIZE//BATCH_SIZE
hidden_unit = 1024
embedding_size = 256
print(BUFFER_SIZE)


dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.batch(BATCH_SIZE)

4211


In [ ]:
tmp_x, tmp_y = next(iter(dataset))
print(tmp_x.shape)
print(tmp_y.shape)

(32, 13)
(32, 13)


In [ ]:
class Encode(tf.keras.Model):
    def __init__(self, embedding_size, vocab_size, hidden_units):
        super(Encode, self).__init__()
        self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.GRU = tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform')
        self.hidden_units = hidden_units
        
    def call(self, x, hidden_state):
        try:
            x = self.Embedding(x)
        except:
            print(x, print(inp_lang.vocab_size))          
        outputs, last_state = self.GRU(x, hidden_state)
        return outputs, last_state
    
    def init_hidden_state(self, batch_size):
        return tf.zeros([batch_size, self.hidden_units])

In [ ]:
encoder = Encode(embedding_size, inp_lang.vocab_size, hidden_unit)
hidden_state = encoder.init_hidden_state(BATCH_SIZE)
tmp_outputs, last_state = encoder(tmp_x, hidden_state)
print(tmp_outputs.shape)
print(last_state.shape)

(32, 13, 1024)
(32, 1024)


In [ ]:
class Attention(tf.keras.Model):
    def __init__(self, hidden_units):
        super(Attention, self).__init__()
        self.W_out_encode = tf.keras.layers.Dense(hidden_unit)
        self.W_state = tf.keras.layers.Dense(hidden_unit)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, encode_outs, pre_state):
        pre_state = tf.expand_dims(pre_state, axis=1)
        pre_state = self.W_state(pre_state)
        encode_outs = self.W_out_encode(encode_outs)
        score = self.V(
            tf.nn.tanh(
                pre_state + encode_outs)
        )
        score = tf.nn.softmax(score, axis=1)
        context_vector = score*encode_outs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, score
    

In [ ]:
attention = Attention(hidden_unit)
context_vector, attention_weight = attention(tmp_outputs, last_state)
print(context_vector.shape, attention_weight.shape)

(32, 1024) (32, 13, 1)


In [ ]:
class Decode(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_units):
        super(Decode, self).__init__()
        self.hidden_units = hidden_units
        self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.Attention = Attention(hidden_units)
        self.GRU = tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        self.Fc = tf.keras.layers.Dense(vocab_size)
            
    def call(self, x, encode_outs, pre_state):
        x = tf.expand_dims(x, axis=1)
        try:
            x = self.Embedding(x)
        except:
            print(x, print(tar_lang.vocab_size))          
        context_vector, attention_weight = self.Attention(encode_outs, pre_state)
        context_vector = tf.expand_dims(context_vector, axis=1)
        gru_inp = tf.concat([x, context_vector], axis=-1)
        out_gru, state = self.GRU(gru_inp)
        out_gru = tf.reshape(out_gru, (-1, out_gru.shape[2]))
        return self.Fc(out_gru), state
    
    
decode = Decode(tar_lang.vocab_size, embedding_size, hidden_unit)
print(last_state.shape, tmp_outputs.shape, tmp_y[:, 0].shape)
decode_out, state = decode(tmp_y[:, 0], tmp_outputs, last_state)

(32, 1024) (32, 13, 1024) (32,)


In [ ]:
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
EPOCHS = 50
optimizer = tf.keras.optimizers.Adam()
encoder = Encode(embedding_size, vocab_size=inp_lang.vocab_size, hidden_units=hidden_unit)
decoder = Decode(vocab_size=tar_lang.vocab_size, embedding_size=embedding_size, hidden_units=hidden_unit)

In [ ]:
for epoch in range(EPOCHS):
    total_loss = 0
    for batch_id, (x, y) in enumerate(dataset.take(N_BATCH)):
        loss = 0
        with tf.GradientTape() as tape:
            first_state = encoder.init_hidden_state(batch_size=BATCH_SIZE)
            encode_outs, last_state = encoder(x, first_state)
            decode_state = last_state
            decode_input = [tar_lang.word2id["<start>"]]*BATCH_SIZE
            
            for i in range(1, y.shape[1]):
                decode_out, decode_state = decoder(decode_input, encode_outs, decode_state)
                loss += loss_function(y[:, i], decode_out)
                decode_input = y[:, i]
                
            train_vars = encoder.trainable_variables + decoder.trainable_variables
            grads = tape.gradient(loss, train_vars)
            optimizer.apply_gradients(zip(grads, train_vars))
        total_loss += loss
    print(total_loss.numpy())

8320.274
7247.522
6657.011
6095.486
5534.8374
4982.254
4532.11
4017.9297
3557.948
3173.6047
2842.0027
2560.5618
2225.467
1944.0653
1756.5061
1531.0287
1314.6042
1126.2614
977.7957
870.2354
755.12695
641.2595
564.043
497.95233
428.9099
375.7786
335.45346
314.78412
280.1253
251.35327
218.20825
205.34764
189.33994
179.93619
179.80649
188.20985
204.65053
228.60097
205.4685
168.55176
144.91571
122.4156
107.89786
100.55751
123.088104
159.07336
152.77591
134.92027
131.08786
150.4461


In [ ]:
def translate(inputs):
    print(inp_lang.vector_to_sentence(inputs[0].numpy()))
    result = ''

    hidden = encoder.init_hidden_state(batch_size=1)
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    print(enc_out.shape, dec_hidden.shape)
    
    dec_input = [tar_lang.word2id['<start>']]
    for t in range(tar_lang.max_len):
        predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += tar_lang.id2word[predicted_id] + ' '
        dec_input = [predicted_id]
    return result
  
for inp, tar in dataset.take(N_BATCH):
    print(translate(inp[1:2,:]))
    break

<start> and the reason is  it aposs been done  <end> <pad>
(1, 13, 1024) (1, 1024)
và lý do là  nó được thực hiện  <end> được thực 


<bound method DatasetV2.take of <BatchDataset element_spec=(TensorSpec(shape=(None, 13), dtype=tf.int32, name=None), TensorSpec(shape=(None, 13), dtype=tf.int32, name=None))>>

In [ ]:
for inp, tar in dataset.take(N_BATCH):
    print(translate(inp[2:3:1]))
    break

<start> and a number of sopranos uploaded their parts  <end> <pad> <pad>
(1, 13, 1024) (1, 1024)
và một số nữ cao đã gửi đoạn của họ  <end> của 


In [ ]:
for inp, tar in dataset.take(N_BATCH):
    print(translate(inp[3:4:1]))
    break

<start> i don apost have anything interesting to say  quot <end> <pad>
(1, 13, 1024) (1, 1024)
chẳng nhẽ chẳng có gì hay ho để nói cả  quot <end> 


In [ ]:
print(translate(inp[30:31,:]))

<start> but the other thing they needed was speed  <end> <pad> <pad>
(1, 13, 1024) (1, 1024)
nhưng điều còn lại họ cần là tốc độ  <end> cần là 


In [ ]:
for inp, tar in dataset.take(N_BATCH):
    print(translate(inp[22:23:1]))
    break

<start> it aposs no different than sending a text  <end> <pad> <pad>
(1, 13, 1024) (1, 1024)
nó không khác gì việc gửi một tin nhắn  <end> việc gửi 


In [ ]:
inp

<tf.Tensor: shape=(32, 13), dtype=int32, numpy=
array([[ 593, 2163, 1373, 5274,  863, 5112,  480,  560, 3599, 3129, 5139,
           1,  328],
       [ 593, 2163, 1373, 2503, 5112,    1, 5700, 5913, 3423, 4567,    1,
         328,    0],
       [ 593, 2163, 3491, 1234, 4023,  904, 1225, 1552,  345,    1,  328,
           0,    0],
       [ 593,  234, 4697, 4070, 1365, 4328, 1287, 5322, 5544,    1, 5645,
         328,    0],
       [ 593, 2163, 3651, 1523, 5700, 3668, 1373,    1, 4988, 1840,    1,
         328,    0],
       [ 593, 5639, 3841, 5112, 3491, 2743, 4023, 3682, 3118, 4170,    1,
         328,    0],
       [ 593, 4950, 4718,    1,  586, 3841, 5877, 5424, 4596, 2486,    1,
         328,    0],
       [ 593, 4390, 3958,   49, 4224, 4023, 5401,    1, 4558, 1110, 4122,
           1,  328],
       [ 593, 5639, 1373, 1359, 5112, 1864, 2967, 3841, 1858,    1,  328,
           0,    0],
       [ 593, 2607, 5877, 1531, 3491, 4078, 3599, 1552, 4975,    1,  328,
           0,    0],
  